# Merging Faces

### Importing libraries

In [1]:
import cv2
import dlib
import numpy
import glob

### Some parameters for the Merging

In [2]:
PREDICTOR_PATH = "shape_predictor_68_face_landmarks.dat"
SCALE_FACTOR = 1 
FEATHER_AMOUNT = 11

FACE_POINTS = list(range(17, 68))
MOUTH_POINTS = list(range(48, 61))
RIGHT_BROW_POINTS = list(range(17, 22))
LEFT_BROW_POINTS = list(range(22, 27))
RIGHT_EYE_POINTS = list(range(36, 42))
LEFT_EYE_POINTS = list(range(42, 48))
NOSE_POINTS = list(range(27, 35))
JAW_POINTS = list(range(0, 17))

# Points used to line up the images.
ALIGN_POINTS = (LEFT_BROW_POINTS + RIGHT_EYE_POINTS + LEFT_EYE_POINTS +
                               RIGHT_BROW_POINTS + NOSE_POINTS + MOUTH_POINTS)

# Points from the second image to overlay on the first. The convex hull of each
# element will be overlaid.
OVERLAY_POINTS = [
    LEFT_EYE_POINTS + RIGHT_EYE_POINTS + LEFT_BROW_POINTS + RIGHT_BROW_POINTS,
    NOSE_POINTS + MOUTH_POINTS,
]

# Amount of blur to use during colour correction, as a fraction of the
# pupillary distance.
COLOUR_CORRECT_BLUR_FRAC = 0.6

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(PREDICTOR_PATH)

### get_landmarks function // Returns Facial landmarks of face images

In [3]:
# class TooManyFaces(Exception):
#     pass

# class NoFaces(Exception):
#     pass

def get_landmarks(im):
    # Returns facial landmarks as (x,y) coordinates
    rects = detector(im, 1)
    
    if len(rects) > 1:
        return "error"
    if len(rects) == 0:
        return "error"

    return numpy.matrix([[p.x, p.y] for p in predictor(im, rects[0]).parts()])

### get_face_mask function // Returns the mask, from the image to be overlayed

In [4]:
def draw_convex_hull(im, points, color):
    points=points.astype(numpy.int32)
    points = cv2.convexHull(points)
    cv2.fillConvexPoly(im, points, color=color)

def get_face_mask(im, landmarks):
    im = numpy.zeros(im.shape[:2], dtype=numpy.float64)

    for group in OVERLAY_POINTS:
        draw_convex_hull(im,
                         landmarks[group],
                         color=1)

    im = numpy.array([im, im, im]).transpose((1, 2, 0))

    im = (cv2.GaussianBlur(im, (FEATHER_AMOUNT, FEATHER_AMOUNT), 0) > 0) * 1.0
    im = cv2.GaussianBlur(im, (FEATHER_AMOUNT, FEATHER_AMOUNT), 0)

    return im

### Function to generate transformation matrix, to align the two images using Procrustes analysis

In [5]:
def transformation_from_points(points1, points2):
    
    # Solve the procrustes problem by subtracting centroids, scaling by the
    # standard deviation, and then using the SVD to calculate the rotation. See
    # the following for more details:
    #   https://en.wikipedia.org/wiki/Orthogonal_Procrustes_problem

    points1 = points1.astype(numpy.float64)
    points2 = points2.astype(numpy.float64)

    c1 = numpy.mean(points1, axis=0)
    c2 = numpy.mean(points2, axis=0)
    points1 -= c1
    points2 -= c2

    s1 = numpy.std(points1)
    s2 = numpy.std(points2)
    points1 /= s1
    points2 /= s2

    U, S, Vt = numpy.linalg.svd(points1.T * points2)
    R = (U * Vt).T

    return numpy.vstack([numpy.hstack(((s2 / s1) * R,
                                       c2.T - (s2 / s1) * R * c1.T)),
                         numpy.matrix([0., 0., 1.])])

### Function that returns the image matrix and its landmarks as an array

In [6]:
def read_im_and_landmarks(image):
    im = image
    im = cv2.resize(im,None,fx=1, fy=1, interpolation = cv2.INTER_LINEAR)
    im = cv2.resize(im, (im.shape[1] * SCALE_FACTOR,
                         im.shape[0] * SCALE_FACTOR))
    s = get_landmarks(im)

    return im, s

### Function that uses the transformation matrix to rotate and align the two faces

In [7]:
def warp_im(im, M, dshape):
    output_im = numpy.zeros(dshape, dtype=im.dtype)
    cv2.warpAffine(im,
                   M[:2],
                   (dshape[1], dshape[0]),
                   dst=output_im,
                   borderMode=cv2.BORDER_TRANSPARENT,
                   flags=cv2.WARP_INVERSE_MAP)
    return output_im

### Function to match the skin tone of two faces

In [8]:
def correct_colours(im1, im2, landmarks1):
    blur_amount = COLOUR_CORRECT_BLUR_FRAC * numpy.linalg.norm(
                              numpy.mean(landmarks1[LEFT_EYE_POINTS], axis=0) -
                              numpy.mean(landmarks1[RIGHT_EYE_POINTS], axis=0))
    blur_amount = int(blur_amount)
    if blur_amount % 2 == 0:
        blur_amount += 1
    im1_blur = cv2.GaussianBlur(im1, (blur_amount, blur_amount), 0)
    im2_blur = cv2.GaussianBlur(im2, (blur_amount, blur_amount), 0)

    # Avoid divide-by-zero errors.
    im2_blur += (128 * (im2_blur <= 1.0)).astype(im2_blur.dtype)

    return (im2.astype(numpy.float64) * im1_blur.astype(numpy.float64) /
                                                im2_blur.astype(numpy.float64))

## Main driving function

In [9]:
def swappy(image1, image2):
       
    im1, landmarks1 = read_im_and_landmarks(image1)
    im2, landmarks2 = read_im_and_landmarks(image2)
    if (landmarks2 == "error"):
        print "No or too many faces"
        return im2

    M = transformation_from_points(landmarks1[ALIGN_POINTS],
                                   landmarks2[ALIGN_POINTS])
    
    mask = get_face_mask(im2, landmarks2)
    warped_mask = warp_im(mask, M, im1.shape)
    combined_mask = numpy.max([get_face_mask(im1, landmarks1), warped_mask],
                              axis=0)

    warped_im2 = warp_im(im2, M, im1.shape)
    warped_corrected_im2 = correct_colours(im1, warped_im2, landmarks1)

    output_im = im1 * (1.0 - combined_mask) + warped_corrected_im2 * combined_mask
    cv2.imwrite('output.jpg', output_im)
    image = cv2.imread('output.jpg')
    return image

## Test function to overlay two images

In [ ]:
   

## Enter the paths to your input images here    
image1 = cv2.imread('Side-bangs-215x300.jpg')
image2 = cv2.imread('PicsArt_10-18-04.44.03.jpg')

swapped1 = swappy(image1, image2)
out = cv2.pyrDown(swapped1)
cv2.imshow("Face Swap 1", swapped1)

cv2.waitKey(0)

cv2.destroyAllWindows()      

### To make an array of images present in the directory

In [ ]:
data = []
files = glob.glob("*.jpg")
for myFile in files:
    print(myFile)
    image = cv2.imread(myFile)
    data.append(image)

for f in range(len(data)):
    image1 = data[f]
    image2 = cv2.imread('Animesh.jpeg') #Enter your image here!

    swapped1 = swappy(image1, image2)
    out = cv2.pyrDown(swapped1)
    cv2.imshow("Face Swap" + str(f), swapped1)

    cv2.waitKey(0)
cv2.destroyAllWindows()    

## Test function no. 3 : Works on webcam (Slow for now, but can be improved)

In [10]:
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    image1 = cv2.imread('longHaristyle.jpg')
    image2 = frame
    swapped1 = swappy(image1, image2)
    out = cv2.pyrDown(swapped1)
    cv2.imshow("Face Swap", out)
    if cv2.waitKey(1) == 13:
        break
cap.release()        
cv2.destroyAllWindows()